In [2]:
import json
from pickle import load
from numpy import array
from numpy import argmax
#from pickle import dumptaged
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences 
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding,LSTM,RepeatVector,TimeDistributed,Dense
from keras.callbacks import ModelCheckpoint
from numpy.random import rand
from numpy.random import shuffle
import unicodedata
import re
import string

In [3]:
#add path to the json file
data_path = 'C:\\Users\\welcome\\Desktop\\PYTHON\\andrew ng\\yugasa\\hinglish to English\\git demo\\English-to-Hinglish-Neural-Machine-Translation\hinglish_upload_v1.json'

In [4]:
data = []  # Empty list to store dictionaries

with open(data_path, 'r') as f:
    for line in f:
        try:
            obj = json.loads(line)
            dictionary = {
              'en': obj['translation']['en'],
              'hi_en': obj['translation']['hi_ng'],
                # Add more key-value pairs as needed
            }
            data.append(dictionary)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Now 'data' contains an array of dictionaries from json file



In [5]:
 del dictionary

In [6]:
print(data[66]['en'])
len(data)

I haven't seen a lot of the old classics for a long time. 


189102

In [7]:
# To seprate the two languages into different lists

en = []
hi_en = []

for dictionary in data:
    # Extract values for key1, key2, and key3
        value1 = dictionary['en']
        value2 = dictionary['hi_en']

        # Append values to respective arrays
        en.append(value1)
        hi_en.append(value2)

      
        

In [8]:
print(type(en[1]))

<class 'str'>


In [9]:
# to check if spliting the data was done correctly
for i in range(3):
    print('English--',en[i])
    print('Hinglish--',hi_en[i])
    print('________________________________________________')
    
print(len(en))
print(len(hi_en))

English-- What's the name of the movie
Hinglish-- film ka kya naam hai
________________________________________________
English-- Hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. 
Hinglish-- namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.
________________________________________________
English-- Do you think you will like the movie
Hinglish-- kya aapako lagata hai ki aapako film pasand aaegee
________________________________________________
189102
189102


In [10]:
#zipping the two list into one numpy array
import numpy as np
pairs= np.array(list(zip(hi_en,en)))


In [11]:
np.set_printoptions(threshold=10000)
print(pairs)

[['film ka kya naam hai' "What's the name of the movie"]
 ['namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.'
  'Hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ']
 ['kya aapako lagata hai ki aapako film pasand aaegee'
  'Do you think you will like the movie']
 ...
 ['Me: Aaj kuch toofani karte hai.\n\nMom: Pani ki bottles bhar ke fridge mein rakh de.'
  "Me: Let's do some stormy today.\n\nMom: Fill the water bottles and keep them in the fridge."]
 ['Pyar mangi to Jaan dengi,milk mango to kher dengi,agaar Kashmir mangi to chir dengi.'
  "Ask for love we'll give life, ask for milk we'll give porridge, ask for Kashmir we'll tear you"]
 ["Ye sab aunty'on ke saath?" 'All this with aunties?']]


In [12]:

#data cleaning
# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = unicodedata.normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

In [13]:

clean_pairs = clean_pairs(pairs)

In [15]:
#  Check if the length of the source sentence (column at index 0) is greater than 0
non_empty_rows = clean_pairs[:, 0] != ''

# Remove the rows where the length is 0
clean_pairs = clean_pairs[non_empty_rows, :]

# Print the removed rows
removed_rows = clean_pairs[~non_empty_rows, 0]
print("Removed rows:", removed_rows)

Removed rows: []


In [16]:
for i, line in enumerate(clean_pairs):
    # Check if the length of the source sentence (column at index 0) is greater than 7
    
    if len(clean_pairs[i, 0]) == 0:
        # Index of the row you want to remove
        print('--',clean_pairs[i, 0])

In [17]:
np.set_printoptions(threshold=100000)
print(clean_pairs)

[['film ka kya naam hai' 'whats the name of the movie']
 ['namaste sada hua tomatoes score mahaan hai lekin meta critic score is gunavatta kee philm se thoda kam lagata hai'
  'hi the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality']
 ['kya aapako lagata hai ki aapako film pasand aaegee'
  'do you think you will like the movie']
 ...
 ['me aaj kuch toofani karte hai mom pani ki bottles bhar ke fridge mein rakh de'
  'me lets do some stormy today mom fill the water bottles and keep them in the fridge']
 ['pyar mangi to jaan dengimilk mango to kher dengiagaar kashmir mangi to chir dengi'
  'ask for love well give life ask for milk well give porridge ask for kashmir well tear you']
 ['ye sab auntyon ke saath' 'all this with aunties']]


In [18]:
def max_length(lines):
	return max(len(line.split()) for line in lines)

print(max_length(clean_pairs[:, 0]))
print(max_length(clean_pairs[:, 1]))

273
247


In [19]:
clean_pairs[:, 0]

array(['film ka kya naam hai',
       'namaste sada hua tomatoes score mahaan hai lekin meta critic score is gunavatta kee philm se thoda kam lagata hai',
       'kya aapako lagata hai ki aapako film pasand aaegee', ...,
       'me aaj kuch toofani karte hai mom pani ki bottles bhar ke fridge mein rakh de',
       'pyar mangi to jaan dengimilk mango to kher dengiagaar kashmir mangi to chir dengi',
       'ye sab auntyon ke saath'], dtype='<U1463')

In [20]:
#settig the sentence lenth 
# Create an empty list to store the indices of rows to delete
rows_to_delete = []

for i, line in enumerate(clean_pairs):
    # Check if the length of the source sentence (column at index 0) is greater than 7
    if len(clean_pairs[i, 0]) > 20:
        # If the condition is met, add the index 'i' to the list of rows to delete
        rows_to_delete.append(i)

# Delete the rows specified by 'rows_to_delete'
new_data = np.delete(clean_pairs, rows_to_delete, axis=0)

In [59]:
print(len(new_data))

12537


In [24]:
for i, line in enumerate(new_data):
    if len(new_data[i, 0]) > 15:
        print('--',new_data[i, 0])

-- film ka kya naam hai
-- film kab banee thee
-- film ka kya naam hai
-- vah majaakiya hai
-- imdb ke alavanahi
-- haa week mei do baar
-- especially films
-- scary good movie hai
-- na gaslight nahi
-- gaslight to dekho
-- hello tum kaise ho
-- tho ache rating hein
-- vaapas usake paas
-- saamagree cheezen
-- me accha hun thanks
-- aur wo funny bhi tha
-- timberlake cool tha
-- winklevoss twins
-- lead actors kon hai
-- kya tumne dekhi hai
-- mein se vahiyaat
-- i think tum sahi ho
-- main ek ladki hoon
-- wo bohat lovely hai
-- ye really achi hai
-- cgi really good hai
-- haan muje patha hein
-- comedy movie hai
-- tim meadows bhi he
-- yeh ek thos film hai
-- unhonne nahin kiya
-- ee tho ek off hi
-- nahi hai mere paas
-- haan tum wahi ho
-- aapka din accha ho
-- no maine nahi dekhi
-- sangharsh kya hai
-- yah ek sangeet hai
-- mujhe nahin pata
-- ajeeb lag raha hai
-- the social network
-- kuch reviews hein
-- ache ratings hein
-- aapka din great ho
-- to story kya hai
-- ise jaan

In [25]:
clean_pairs = new_data
print(len(clean_pairs))


12537


In [62]:
def save_clean_data(sentences, filename):
 dump(sentences, open(filename, 'wb'))
 print('Saved: %s' % filename)
 

save_clean_data(clean_pairs, 'english-hi.pkl')

Saved: english-hi.pkl


In [22]:
# load a clean dataset
def load_clean_sentences(filename):
 return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
 dump(sentences, open(filename, 'wb'))
 print('Saved: %s' % filename)
 
# load dataset
raw_dataset = load_clean_sentences('english-hi.pkl')
 
# reduce dataset size
n_sentences = 8000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:7200], dataset[7200:]
# save
save_clean_data(dataset, 'english-hi-both.pkl')
save_clean_data(train, 'english-hi-train.pkl')
save_clean_data(test, 'english-hi-test.pkl')

Saved: english-hi-both.pkl
Saved: english-hi-train.pkl
Saved: english-hi-test.pkl


def load_clean_sentences(filename):
 return load(open(filename, 'rb'))

del en
del hi_en
del data
del clean_pairs

# Train Neural Translation Model


In [64]:
 # load datasets
dataset = load_clean_sentences('english-hi-both.pkl')
train = load_clean_sentences('english-hi-train.pkl')
test = load_clean_sentences('english-hi-test.pkl')

In [65]:
print(type(dataset[1,1]))
print(type(test[1,1]))
print(type(train[1,1]))
import sys
print(sys.getsizeof(dataset[1,1]))
print(len(dataset))
print(len(train))
print(len(test))

<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
98
8000
7200
800


In [ ]:
dataset

array([['timer ko delete kare', 'delete cookie timer'],
       ['beatles ko sunen', 'listen to beatles'],
       ['alarm cancel kare', 'cancel alarm'],
       ['alarms set kare', 'set alarms'],
       ['ke liye mausam', 'weather for the'],
       ['message send karo', 'send a message'],
       ['alarm ko on karo', 'turn on alarm'],
       ['kya aj dhoop hogi', 'will it be sunny today'],
       ['kya bahar sunny hai', 'is it sunny out'],
       ['timer ko pause kare', 'pause the timer'],
       ['timer ko cancel karo', 'cancel timer'],
       ['timer ko cancel karo', 'cancel the timer'],
       ['traffic kya hai', 'what s the traffic'],
       ['playlist me add karo', 'add to playlist'],
       ['ye kitaab', 'the notebook'],
       ['chats ko open kare', 'open chats'],
       ['seconds ko band karo', 'take off seconds'],
       ['rihanna ko play kare', 'play rihanna'],
       ['timer ko reset karen', 'reset the timer'],
       ['track ko skip kare', 'skip track'],
       ['yah to bland 

In [24]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [68]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [69]:
# prepare english tokenizer
hi_tokenizer = create_tokenizer(dataset[:, 0])
hi_vocab_size = len(hi_tokenizer.word_index) + 1
hi_length = max_length(dataset[:, 0])
print('Hinglish Vocabulary Size: %d' % hi_vocab_size)
print('Hinglish Max Length: %d' % (hi_length))
# prepare german tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 1])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 1])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

Hinglish Vocabulary Size: 2100
Hinglish Max Length: 6
English Vocabulary Size: 1807
English Max Length: 13


In [26]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [27]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [72]:
# prepare training data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 1])
trainY = encode_sequences(hi_tokenizer, hi_length, train[:, 0])
trainY = encode_output(trainY, hi_vocab_size)
# prepare validation data
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 1])
testY = encode_sequences(hi_tokenizer, hi_length, test[:, 0])
testY = encode_output(testY, hi_vocab_size)

In [73]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# define model
model = define_model(eng_vocab_size, hi_vocab_size, eng_length, hi_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13, 256)           462592    
                                                                 
 lstm_4 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_2 (RepeatVec  (None, 6, 256)            0         
 tor)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 6, 256)            525312    
                                                                 
 time_distributed_2 (TimeDi  (None, 6, 2100)           539700    
 stributed)                                                      
                                                                 
Total params: 2052916 (7.83 MB)
Trainable params: 2052

In [74]:
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Epoch 1/30

Epoch 1: val_loss improved from inf to 3.16296, saving model to model.h5
113/113 - 42s - loss: 4.0129 - val_loss: 3.1630 - 42s/epoch - 370ms/step
Epoch 2/30

Epoch 2: val_loss improved from 3.16296 to 2.81677, saving model to model.h5
113/113 - 32s - loss: 2.9020 - val_loss: 2.8168 - 32s/epoch - 279ms/step
Epoch 3/30

Epoch 3: val_loss improved from 2.81677 to 2.54945, saving model to model.h5
113/113 - 31s - loss: 2.5718 - val_loss: 2.5494 - 31s/epoch - 277ms/step
Epoch 4/30

Epoch 4: val_loss improved from 2.54945 to 2.35021, saving model to model.h5
113/113 - 30s - loss: 2.3094 - val_loss: 2.3502 - 30s/epoch - 266ms/step
Epoch 5/30

Epoch 5: val_loss improved from 2.35021 to 2.20002, saving model to model.h5
113/113 - 31s - loss: 2.0961 - val_loss: 2.2000 - 31s/epoch - 277ms/step
Epoch 6/30

Epoch 6: val_loss improved from 2.20002 to 2.06280, saving model to model.h5
113/113 - 31s - loss: 1.9108 - val_loss: 2.0628 - 31s/epoch - 272ms/step
Epoch 7/30

Epoch 7: val_loss im

# Evaluate Neural Translation Model


In [28]:
...
# load datasets
dataset = load_clean_sentences('english-hi-both.pkl')
train = load_clean_sentences('english-hi-train.pkl')
test = load_clean_sentences('english-hi-test.pkl')
# prepare english tokenizer
hi_tokenizer = create_tokenizer(dataset[:, 0])
hi_vocab_size = len(hi_tokenizer.word_index) + 1
hi_length = max_length(dataset[:, 0])
# prepare german tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 1])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 1])
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 1])

In [29]:
# load model
model = load_model('model.h5')

In [30]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [31]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [61]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, hi_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
        



# calculate BLEU score
#print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
#print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
#print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
#print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
def evaluate_model(model, tokenizer, sources, raw_dataset):
 actual, predicted = list(), list()
 for i, source in enumerate(sources):
 # translate encoded source text
 source = source.reshape((1, source.shape[0]))
 translation = predict_sequence(model, eng_tokenizer, source)
 raw_target, raw_src = raw_dataset[i]
 if i < 10:
 print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
 actual.append([raw_target.split()])
 predicted.append(translation.split())
 # calculate BLEU score
 print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
 print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
 print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
 print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [59]:
# load datasets
dataset = load_clean_sentences('english-hi-both.pkl')
train = load_clean_sentences('english-hi-train.pkl')
test = load_clean_sentences('english-hi-test.pkl')
# prepare english tokenizer
hi_tokenizer = create_tokenizer(dataset[:, 0])
hi_vocab_size = len(hi_tokenizer.word_index) + 1
hi_length = max_length(dataset[:, 0])
# prepare german tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 1])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 1])
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 1])

In [103]:
# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, hi_tokenizer, trainX, train)

train
src=[events in concord], target=[concord mei events], predicted=[movie mei please]
src=[repeat song], target=[song ko repeat kare], predicted=[song ko repeat kare]
src=[pause timer please], target=[timer pause kare], predicted=[timer pause kare]
src=[traffic report], target=[traffic report], predicted=[traffic love]
src=[the imitation game], target=[nakalee khel], predicted=[bot me me he]
src=[message i can t get off work], target=[i can t get off work], predicted=[kya alarm alarm alarm hai]
src=[monsters university], target=[monsters university], predicted=[fall ke]
src=[only country], target=[sirf country hai], predicted=[kya abhi acha hai]
src=[go away alarm], target=[alarm ko band karo], predicted=[ek alarm alarm lagaye]
src=[will it rain today], target=[kya aj barish hogi], predicted=[kya aj baarish hogi]



KeyboardInterrupt



In [108]:
def translate_sentence(model, tokenizer, sentence):
    # Tokenize the source sentence
    source_seq = tokenizer.texts_to_sequences([sentence])[0]
    source_seq = np.array(source_seq).reshape((1, len(source_seq)))

    # Use the predict_sequence function to generate the translation
    translation = predict_sequence(model, tokenizer, source_seq)

    return translation

# Example usage:
source_sentence = "song ko repeat karee"
translation = translate_sentence(model, eng_tokenizer, source_sentence)
print(f'Source: {source_sentence}')
print(f'Translation: {translation}')


Source: song ko repeat karee
Translation: what the timer


In [80]:
print(list(eng_tokenizer.word_index.keys()))


['the', 'is', 'timer', 'it', 'alarm', 'play', 'today', 'song', 'what', 'will', 'weather', 'music', 'stop', 'off', 'how', 'turn', 's', 'to', 'pause', 'next', 'when', 'traffic', 'on', 'be', 'skip', 'a', 'i', 'this', 'set', 'in', 'rain', 'please', 'alarms', 'like', 'my', 'tomorrow', 'start', 'time', 'cancel', 'events', 'track', 'hello', 'you', 'cold', 'snooze', 'hot', 'there', 'outside', 'up', 'are', 'me', 'show', 'new', 'now', 'hi', 'snow', 'some', 'yes', 'repeat', 'change', 'that', 'sunny', 'text', 'an', 'send', 'at', 'songs', 'for', 'pm', 'shut', 'chats', 'delete', 'whats', 'reset', 'add', 'out', 'message', 'forecast', 'go', 'temperature', 'get', 'high', 'rock', 'all', 'end', 'going', 'much', 'no', 'concerts', 'from', 'have', 'make', 'current', 'good', 'oh', 'not', 'resume', 'yeah', 'right', 'restart', 'minutes', 'windy', 'am', 'too', 'radio', 'top', 'pop', 'more', 'of', 'do', 'parties', 'left', 'hey', 'does', 'reply', 'sunrise', 'jazz', 'movie', 'rap', 'was', 'playlist', 'quiet', 'day

In [84]:
# test on some test sequences
print('test')
evaluate_model(model, hi_tokenizer, testX, test)

test
src=[stop the alarm], target=[alarm ko stop kare], predicted=[alarm ko stop kare]
src=[hello], target=[hello], predicted=[hello]
src=[show alarms i have], target=[mere alarms dikhao], predicted=[mere ke liye]
src=[hi], target=[hi], predicted=[hi]
src=[alarm in], target=[me alarm], predicted=[ghante ka]
src=[i am not there], target=[mai vaha nahi hu], predicted=[mai yaad nahi hu]
src=[okay why did you leave], target=[ok aap kyu gaye the], predicted=[free reviews bhi]
src=[hello], target=[hello], predicted=[hello]
src=[skip to the next song], target=[agle song par jaayen], predicted=[agle song par jaayen]
src=[repeat track], target=[track ko repeat kare], predicted=[track ko repeat kare]



KeyboardInterrupt

